Here we are going to explore the connectivity between objects by creating a map with the closest object

In [39]:
%matplotlib qt
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.animation as animation
from datetime import timedelta
import xarray as xr
from explfunctions import coraldistancemap, followPath
import cmocean
import seaborn as sns
from matplotlib.colors import ListedColormap
from matplotlib import cm

In [2]:
name = 'output-corals-regridded'
fb = 'forward' #variable to determine whether the flowfields are analysed 'forward' or 'backward' in time
runtime=timedelta(seconds=44)                            # total time to execute the particleset
outputdt = timedelta(seconds=0.1)
filename = name+'.nc'
dfilename = 'd2c'+name+fb+str(runtime.seconds)+'.nc'
flowdata = xr.open_dataset(filename)
data = xr.open_dataset(dfilename)

In [3]:
objects = np.load('objects.npy')
coastmask = np.load('coastmask.npy')
xmesh,ymesh = np.meshgrid(flowdata['X'],flowdata['Y'])
umask = np.ma.masked_invalid(flowdata['U'][0])

In [4]:
mat = np.zeros(objects.shape)
coralmap = np.zeros((len(objects)+1,len(objects[0]),len(objects[0,0])))
for i in range(len(objects)):
    mat,coralmap[i] = coraldistancemap(umask.mask,objects[i],xmesh,ymesh)
coralmap[-1] = np.ones(xmesh.shape)*9998

In [5]:
# index = np.ma.masked_array(np.argmin(coralmap,axis=0),umask.mask)
# np.save('closestobject',np.argmin(coralmap,axis=0),allow_pickle=True)
closestobject = np.load('closestobject.npy')
index = np.ma.masked_array(closestobject,umask.mask)

In [73]:
objects.shape

(22, 100, 900)

In [7]:
%matplotlib qt

from random import shuffle
palette = sns.hls_palette(len(objects)+1, l=.7, s=.8)
shuffle(palette)
my_cmap = ListedColormap(palette)

fig = plt.figure(figsize=(18,4))
ax = plt.axes()
pc = ax.pcolormesh(xmesh,ymesh,index,cmap=my_cmap)
for i in range(len(objects)):
#     ax.scatter(xmesh[objects[i]],ymesh[objects[i]],s=10,c=[palette[i]])
    ax.text(np.mean(xmesh[objects[i]]),np.mean(ymesh[objects[i]]),str(i),horizontalalignment='center', verticalalignment='top')
# ax.invert_yaxis()
ax.set_xlabel("x [m]")
ax.set_ylabel("height [m]")
# plt.savefig('Figures/closestobject')

Text(0, 0.5, 'height [m]')

Now we have information on the distance to the nearest object and which object is nearest. Now we want to compute the probability that particles in one halo enter another halo. For this we first need to figure out which particles pass through a single halo. The easiest way to do this is to sample the closest object.

In [8]:
objectnr = 5
d2cmax = 0.05
objectdata = data.where(data['closestobject'][:,0]==objectnr,drop=True)
objectdata = objectdata.where(objectdata['d2c'][:,0]<d2cmax,drop=True)

In [9]:
fig = plt.figure(figsize=(18,4))
ax = plt.axes()
ax.set_facecolor('k')
cf = ax.contourf(flowdata['X'],flowdata['Y'],flowdata['U'][8,:,:],np.linspace(-10,-9,21),extend='both',cmap=cmocean.cm.dense_r)
sc = ax.scatter(objectdata['lon'][:,0], -objectdata['z'][:,0], c=np.min(objectdata['d2c'],axis=1),s=10, marker="o",cmap=cmocean.cm.amp_r)
time_text = ax.text(0.05, 0.95,'',horizontalalignment='left',verticalalignment='top', transform=ax.transAxes)
cbar = fig.colorbar(sc,orientation= 'horizontal',label='Smallest distance to coral [m]')

def animate(i,fig,sc):
    sc.set_offsets(np.c_[objectdata['lon'][:,i], -objectdata['z'][:,i]])
    ts = i*outputdt.microseconds/1000000
    time_text.set_text('time = %.1f seconds' % ts)
    return sc, time_text,

ax.set_xlabel("x [m]")
ax.set_ylabel("z (m)")
ax.set_xlim(-0.5,8.5)
ax.set_ylim(-0.49,0.5)
anim = animation.FuncAnimation(fig, animate, fargs = (fig, sc),
                               frames = len(objectdata['lon'][0]), blit=True)
# anim.save('Figures/d2clonc'+filename+'-'+str(runtime.seconds)+'-'+fb+str(d2cmax)+'.mp4')
plt.show()

We have now selected the particles that start near a single coral object. The next two steps are figuring out how to select all particles that pass through the halo of the single object and start looking at connectivity between objects

In [49]:
objectnr = 8
d2cmax = 0.05
objectdata = data.where(data['closestobject'][:,0]==objectnr,drop=True)
objectdata = objectdata.where(objectdata['d2c'][:,0]<d2cmax,drop=True)

In [50]:
fig = plt.figure(figsize=(18,4))
ax = plt.axes()
ax.set_facecolor('k')
cf = ax.contourf(flowdata['X'],flowdata['Y'],flowdata['U'][8,:,:],np.linspace(-10,-9,21),extend='both',cmap=cmocean.cm.dense_r)
sc = ax.scatter(objectdata['lon'][:,0], -objectdata['z'][:,0], c=np.min(objectdata['d2c'],axis=1),s=10, marker="o",cmap=cmocean.cm.amp_r)
time_text = ax.text(0.05, 0.95,'',horizontalalignment='left',verticalalignment='top', transform=ax.transAxes)
cbar = fig.colorbar(sc,orientation= 'horizontal',label='Smallest distance to coral [m]')

def animate(i,fig,sc):
    sc.set_offsets(np.c_[objectdata['lon'][:,i], -objectdata['z'][:,i]])
    ts = i*outputdt.microseconds/1000000
    time_text.set_text('time = %.1f seconds' % ts)
    return sc, time_text,

ax.set_xlabel("x [m]")
ax.set_ylabel("z (m)")
ax.set_xlim(-0.5,8.5)
ax.set_ylim(-0.49,0.5)
anim = animation.FuncAnimation(fig, animate, fargs = (fig, sc),
                               frames = len(objectdata['lon'][0]), blit=True)
# anim.save('Figures/object'+str(objectnr)+'-'+filename+'-'+str(runtime.seconds)+'-'+fb+str(d2cmax)+'.mp4')
plt.show()

In [51]:
objectdata['closestobject']

<xarray.DataArray 'closestobject' (traj: 314, obs: 441)>
array([[ 8.,  8.,  8., ...,  3.,  3., 15.],
       [ 8.,  8.,  8., ..., 22., 22., 22.],
       [ 8.,  8.,  8., ..., 22., 22., 22.],
       ...,
       [ 8.,  8.,  8., ...,  7.,  7.,  7.],
       [ 8.,  8.,  8., ...,  7.,  7.,  7.],
       [ 8.,  8.,  8., ...,  7.,  7.,  7.]], dtype=float32)
Dimensions without coordinates: traj, obs
Attributes:
    long_name:      
    standard_name:  closestobject
    units:          unknown

In [52]:
connectivity = np.zeros(len(objects))
for i in range(len(objects)):
    print(i)
    if i != objectnr:
        condata = objectdata.where(objectdata['closestobject']==i,drop=True)
        condata = condata.where(condata['d2c']<d2cmax)
        connectivity[i] = len(condata['closestobject'])/len(objectdata['closestobject'])
    else:
        connectivity[i] = 1

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21


In [53]:
condata = objectdata.where(objectdata['closestobject']==22,drop=True)
condata = condata.where(condata['d2c']<d2cmax)
len(condata['closestobject'])

163

In [54]:
from random import shuffle
palette = sns.hls_palette(len(objects), l=.7, s=.8)
shuffle(palette)
my_cmap = ListedColormap(palette)

fig = plt.figure(figsize=(18,4))
ax = plt.axes()
pc = ax.pcolormesh(xmesh,ymesh,index,cmap=my_cmap)
for i in range(len(objects)):
#     ax.scatter(xmesh[objects[i]],ymesh[objects[i]],s=10,c=[palette[i]])
    ax.text(np.mean(xmesh[objects[i]]),np.mean(ymesh[objects[i]]),str(i),horizontalalignment='center', verticalalignment='top')
    ax.text(np.mean(xmesh[objects[i]]),np.mean(ymesh[objects[i]])-0.05,str(round(connectivity[i],2)),horizontalalignment='center', verticalalignment='top')
# ax.invert_yaxis()
ax.set_xlabel("x [m]")
ax.set_ylabel("height [m]")

Text(0, 0.5, 'height [m]')

In [55]:
colors =cmocean.cm.amp
colors(0.3)

fig = plt.figure(figsize=(18,4))
ax = plt.axes()
# pc = ax.pcolormesh(xmesh,ymesh,np.ma.masked_array(np.ones(index.shape)*connectivity[0],~objects[0]),cmap=my_cmap)
for i in range(len(objects)):
#     pc = ax.pcolormesh(xmesh,ymesh,np.ma.masked_array(np.ones(index.shape)*connectivity[i],~objects[i]),cmap=cmocean.cm.amp_r)
    ax.scatter(xmesh[objects[i]],ymesh[objects[i]],s=10,c=[colors(connectivity[i])]*len(xmesh[objects[i]]))
    ax.text(np.mean(xmesh[objects[i]]),np.mean(ymesh[objects[i]]),str(i),horizontalalignment='center', verticalalignment='top')
    ax.text(np.mean(xmesh[objects[i]]),np.mean(ymesh[objects[i]])-0.05,str(round(connectivity[i],2)),horizontalalignment='center', verticalalignment='top')
# ax.invert_yaxis()
ax.set_xlabel("x [m]")
ax.set_ylabel("height [m]")

Text(0, 0.5, 'height [m]')

In [46]:
ymesh[objects[0]].shape

(230,)